In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
! pip install torch
! pip install datasets
! pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
# multinews = load_dataset("multi_news", split="test")
dataset = load_dataset('cnn_dailymail', '3.0.0')


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# for cnn
trn = dataset["train"]
val = dataset['validation']
test = dataset['test']


In [ ]:
trn1 = trn.to_pandas()
val1 = val.to_pandas()
test1 = test.to_pandas()

In [ ]:
# trn1 = trn1.iloc[:int(len(trn1)/10)]
# val1 = val1.iloc[:int(len(val1)/10)]
# test1 = test1.iloc[:int(len(test1)/10)]

trn1 = trn1.iloc[:250]
val1 = val1.iloc[:25]
test1 = test1.iloc[:25]

In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
trn1 = Dataset.from_pandas(trn1)
test1 = Dataset.from_pandas(test1)
val1 = Dataset.from_pandas(val1)

In [ ]:
dataset_dict = DatasetDict({'train': trn1, 'validation': val1, 'test':test1})

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")

In [ ]:
# prefix = "summarize: "

# def preprocess_function(examples):
#     inputs = [prefix + doc for doc in examples["document"]]
#     model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
#     labels = tokenizer(text=examples["summary"], max_length=128, truncation=True)
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs


prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text=examples["highlights"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized = dataset_dict.map(preprocess_function, batched=True)
# tokenized_trn = trn1.map(preprocess_function, batched=True)
# tokenized_val = val1.map(preprocess_function, batched=True)

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='t5-small')
# model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='facebook/bart-large')
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")
model_name = 'facebook/bart-large'

In [ ]:
! pip install torch==2.1.1 torchvision==0.16.1 torchaudio===2.1.1 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
training_args = Seq2SeqTrainingArguments(
output_dir="./results",
evaluation_strategy="epoch",
learning_rate=2e-5,
per_device_train_batch_size=10,
per_device_eval_batch_size=10,
weight_decay=0.01,
save_total_limit=3,
num_train_epochs=10,
logging_steps = 10,
fp16=False,
)


In [ ]:
# trainer = Seq2SeqTrainer(
# model=model,
# args=training_args,
# train_dataset=tokenized_multinews["train"],
# eval_dataset=tokenized_multinews["test"],
# tokenizer=tokenizer,
# data_collator=data_collator,
# )

trainer = Seq2SeqTrainer(
model=model,
args=training_args,
train_dataset=tokenized["train"],
eval_dataset=tokenized["test"],
tokenizer=tokenizer,
data_collator=data_collator,
)


In [ ]:
trainer.train()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
document = "National Archives Yes, it's that time again, folks. It's the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you're here, why don't you sign up to follow us on Twitter. Enjoy the show. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may not be growing fast enough to sustain robust job growth. The unemployment rate dipped, but mostly because more Americans stopped looking for work. The Labor Department says the economy added 120,000 jobs in March, down from more than 200,000 in each of the previous three months. The unemployment rate fell to 8.2 percent, the lowest since January 2009. The rate dropped because fewer people searched for jobs. The official unemployment tally only includes those seeking work. The economy has added 858,000 jobs since December _ the best four months of hiring in two years. But Federal Reserve Chairman Ben Bernanke has cautioned that the current hiring pace is unlikely to continue without more consumer spending."
human_summary = """– The unemployment rate dropped to 8.2% last month, but the economy only added 120,000 jobs, when 203,000 new jobs had been predicted, according to today's jobs report. Reaction on the Wall Street Journal's MarketBeat Blog was swift: "Woah!!! Bad number." The unemployment rate, however, is better news; it had been expected to hold steady at 8.3%. But the AP notes that the dip is mostly due to more Americans giving up on seeking employment."""

In [ ]:
def predict_summary(document):
  device = model.device
  tokenized = tokenizer([document], truncation =True, padding ='longest',return_tensors='pt')
  tokenized = {k: v.to(device) for k, v in tokenized.items()}
  tokenized_result = model.generate(**tokenized, max_length=128)
  tokenized_result = tokenized_result.to('cpu')
  predicted_summary = tokenizer.decode(tokenized_result[0])
  return predicted_summary

In [ ]:
predicted_summary = predict_summary(document)
predicted_summary

'<pad><extra_id_0> New York Time: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times: New York Times:</s>'

In [ ]:
from rouge import Rouge

def get_rouge_scores(actual_summary, predicted_summary):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_summary, actual_summary)
    return [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']]

In [ ]:
# rouge score of validation data

from tqdm import tqdm

rouge1_scores = []
rouge2_scores = []
rougel_scores = []

pred_summary_list = []

for i in tqdm(range(len(val1))):

  doc = val1[i]['article']
  pred_summary = predict_summary(doc)
  human_summary = val1[i]['highlights']

  score = get_rouge_scores(human_summary, pred_summary)

  rouge1_scores.append(score[0])
  rouge2_scores.append(score[1])
  rougel_scores.append(score[2])

  pred_summary_list.append(pred_summary)
  val1[i]["pred_summary"] = pred_summary
  val1[i]['rouge1'] = rouge1_scores
  val1[i]['rouge2'] = rouge2_scores
  val1[i]['rougel'] = rougel_scores

# val1["pred_summary"] = pred_summary_list



val1

100%|██████████| 25/25 [00:31<00:00,  1.26s/it]


Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 25
})

In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.DataFrame(data=val1)

In [ ]:
# rouge score of validation data

from tqdm import tqdm

rouge1_scores = []
rouge2_scores = []
rougel_scores = []

pred_summary_list = []

for i in tqdm(range(len(dataframe))):

  doc = dataframe.iloc[i]['article']
  pred_summary = predict_summary(doc)
  human_summary = dataframe.iloc[i]['highlights']

  score = get_rouge_scores(human_summary, pred_summary)
  if i % 4 == 0:
    print(doc)
    print(human_summary)
    print(pred_summary)

  rouge1_scores.append(score[0])
  rouge2_scores.append(score[1])
  rougel_scores.append(score[2])

  pred_summary_list.append(pred_summary)

dataframe["pred_summary"] = pred_summary_list
dataframe['rouge1'] = rouge1_scores
dataframe['rouge2'] = rouge2_scores
dataframe['rougel'] = rougel_scores

# val1["pred_summary"] = pred_summary_list



dataframe

In [ ]:
dataframe

NameError: ignored

In [ ]:
# average rouge 2
dataframe['rouge2'].mean()

0.09907086393397786